In [1]:
import torch

if torch.cuda.is_available():
    print("CUDA is available. GPU in use:", torch.cuda.get_device_name(0))
else:
    print("CUDA is NOT available. Using CPU.")

CUDA is NOT available. Using CPU.


In [ ]:
import torch
torch.cuda.empty_cache()

### Loading the model and transcribing the sample_audio

In [1]:
# c2t model loading example
import whisper_s2t
from whisper_s2t.backends.ctranslate2.model import BEST_ASR_CONFIG


# model_kwargs = {
#     'device': 'cpu',  # Use 'cpu' if you don't have a GPU
#     'compute_type': 'int8', # Note int8 is only supported for CTranslate2 backend, for others only float16 is supported for lower precision.
#     'asr_options': BEST_ASR_CONFIG
# }



ct2_model = whisper_s2t.load_model(
    model_identifier="large-v2", 
    backend='CTranslate2', 
    compute_type='int8',  # Best for cpu
    device='cpu',
    asr_options=BEST_ASR_CONFIG,
)

'ffmpeg' is not built with soxr resampler, using 'swr' resampler. This may degrade performance.


e:\anaconda3\envs\mals2t\lib\site-packages\ctranslate2\__init__.py:8: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

e:\anaconda3\envs\mals2t\lib\site-packages\whisper_s2t\speech_segmenter\frame_vad.py:74: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @torch.cuda.amp.autocast()
e:\anaconda3\envs\mals2t\lib\site-packages\torch\amp\autocast_mode.py:266: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


In [19]:
import importlib
import split_and_preprocess
importlib.reload(split_and_preprocess)
from split_and_preprocess import process_chunks


output_path = "output.wav"
audio_path = "test.wav"

# Process and merge audio chunks
merged_audio, sr = process_chunks(audio_path, output_path, chunk_duration=30)

Loaded audio shape: (273600,), Sample Rate: 48000
Estimated speech rate: 4.39 onsets/sec
Speech rate is normal.
Original length: 273600, Processed length: 273600
RMS dB level: -17.59 dBFS
Saved merged processed audio to output.wav


In [ ]:
lang_codes = ['ml']
initial_prompts = [None]
# Direct Malayalam-to-English translation
tasks = ['translate']

output_path = "output.wav"
files = [output_path]
    
ct2_out = ct2_model.transcribe_with_vad(
        files,
        lang_codes=lang_codes,
        tasks=tasks,
        initial_prompts=initial_prompts,
        batch_size=16
    )

    # cleaned_segments = [
    #     seg.text for seg in segments
    #     if getattr(seg, "avg_logprob", None) is not None and seg.avg_logprob > -1.0 and getattr(seg, "no_speech_prob", 1.0) < 0.5
    # ]
    # cleaned_transcript = " ".join(cleaned_segments)

print("c2t English translation:")
print(ct2_out[0][0])   # English text

Transcribing: 100%|██████████| 100/100 [01:04<00:00,  1.55it/s]

c2t English translation:
[[{'text': 'You may have all the answers to my questions.', 'avg_logprob': -0.7329445535486395, 'no_speech_prob': 0.19561713933944702, 'start_time': np.float64(0.9), 'end_time': np.float64(5.12)}]]


### calling Gemma 7b to further improve translation

In [4]:
# pip install bitsandbytes accelerate
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(load_in_8bit=True)

tokenizer = AutoTokenizer.from_pretrained("google/gemma-7b-it")
model = AutoModelForCausalLM.from_pretrained("google/gemma-7b-it", quantization_config=quantization_config)

# Craft a prompt with instructions and context
input_text = (
    "You are an expert translator. "
    "Given the following sentence, make it more sensible and fluent in English. "
    "If the sentence does not make sense, use the words in the sentence to form the most plausible translation. "
    "Here is the sentence:\n"
    f"{whisper_output}\n"
    "Improved translation:"
)
# Generate the improved translation
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")
outputs = model.generate(**input_ids)
improved_translation = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(tokenizer.decode(outputs[0]))


KeyboardInterrupt: 